In [16]:
import polars as pl
import pandas as pd

In [17]:
data_df = pd.read_parquet("data/data.parquet")
ukazatel = data_df.loc[data_df["kodukaz"] == 10000]
rocniky = ukazatel["rok"].unique()
print(min(list(rocniky)))

2000


## datové matice ukazatelů v letech

In [49]:
df = pl.read_parquet("data/data.parquet")
ukazatele = pl.read_csv("data/cis_ukazatelu.csv")
uzemi = pl.read_csv("data/cis_uzemi.csv")
df.head()

rok,kodukaz,koduzemi,hodnota
i32,i32,i32,f32
2021,10000,500011,0.0
2021,10000,500020,0.0
2021,10000,500046,0.0
2021,10000,500062,0.0
2021,10000,500071,0.0


In [32]:
uzemi_enum = df["koduzemi"].unique()
matrix_alluzemi = df.pivot(values="hodnota", index=("rok", "koduzemi"), columns="kodukaz", aggregate_function=None).lazy()
matrices = {}
matrixes_uzemi = pl.collect_all([matrix_alluzemi.filter(pl.col("koduzemi") == uzemi) for uzemi in uzemi_enum])
len(matrixes_uzemi)

6272

In [46]:
matrixes_uzemi[0]

rok,koduzemi,10000,10300,10350,10360,10370,10380,10390,10391,40300,40310,40320,40331,40400,40410,40420,40500,40510,40520,40540,40550,40560,40570,40580,40590,40731,40733,40735,40737,40741,40743,40745,40747,40751,40753,40755,…,402505,402602,402604,402605,402606,110960,140800,250100,250110,250120,250130,250140,250150,250160,250170,250180,250190,250200,250210,250220,250230,250240,250250,250260,250270,250280,250290,250300,250310,250320,250330,250340,250350,250360,250370,250380,250390
i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2021,500011,0.0,1602.589355,1.0,1.0,1.0,0.0,0.0,2.0,1.0,103.0,4.0,7.6,1.0,20.0,1.0,1.0,358.0,18.0,0.0,0.0,0.0,10.7,13.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022,500011,0.0,1602.589355,1.0,1.0,1.0,0.0,0.0,2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,500011,0.0,1602.589233,1.0,1.0,1.0,0.0,0.0,2.0,1.0,102.0,4.0,7.6,0.0,0.0,0.0,1.0,325.0,18.0,0.0,0.0,0.0,11.9,13.5,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,500011,0.0,1602.589233,1.0,1.0,1.0,0.0,0.0,2.0,1.0,79.0,3.0,5.8,0.0,0.0,0.0,1.0,337.0,17.0,0.0,0.0,0.0,9.4,14.2,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018,500011,0.0,1602.589233,1.0,1.0,1.0,0.0,0.0,2.0,1.0,81.0,3.0,6.0,0.0,0.0,0.0,1.0,321.0,17.0,0.0,0.0,0.0,9.7,13.6,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2017,500011,0.0,1602.588867,1.0,1.0,1.0,0.0,0.0,2.0,1.0,84.0,3.0,6.0,0.0,0.0,0.0,1.0,301.0,16.0,0.0,0.0,0.0,10.7,11.3,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2016,500011,0.0,1602.588867,1.0,1.0,1.0,0.0,0.0,2.0,1.0,84.0,3.0,6.0,0.0,0.0,0.0,1.0,285.0,15.0,0.0,0.0,0.0,9.0,9.5,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2015,500011,0.0,1602.576416,1.0,1.0,1.0,0.0,0.0,2.0,1.0,84.0,3.0,6.0,0.0,0.0,0.0,1.0,272.0,14.0,0.0,0.0,0.0,8.8,9.3,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2014,500011,0.0,1602.576416,1.0,1.0,1.0,0.0,0.0,2.0,1.0,84.0,3.0,6.0,0.0,0.0,0.0,1.0,259.0,12.0,0.0,0.0,0.0,8.0,7.8,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [50]:
matrixes_uzemi = {m.lazy().select(pl.col("koduzemi")).head(1).collect()[0, 0]: m.drop("koduzemi") for m in matrixes_uzemi}
matrixes_uzemi.keys()

AttributeError: 'int' object has no attribute 'lazy'

In [54]:
import plotly.express as px
import plotly.graph_objects as go

uzemiId = 500011
matrix = matrixes_uzemi[uzemiId].to_pandas()
uzemi_nazev = uzemi.filter(pl.col("koduzemi") == uzemiId).get_column("obec")[0]
ukazatel = 10300
ukazatel_nazev = ukazatele.filter(pl.col("kodukaz") == ukazatel).select("nazev").to_pandas()["nazev"][0]
px.line(matrix, x="rok", y=str(ukazatel), title=f"{uzemi_nazev}: {ukazatel_nazev}")

In [22]:
# df = df.filter(pl.col("rok") == 2021)
# df.head()
for uzemi in df["koduzemi"].unique().to_list():
    print(uzemi)
    df_uzemi = df.filter(pl.col("koduzemi") == uzemi)
    matrix_uzemi = df_uzemi.pivot("rok", "kodukaz", "hodnota")
    print(matrix_uzemi)
    break

500011


ComputeError: found multiple elements in the same group, please specify an aggregation function

rok,kodukaz,koduzemi,hodnota
i32,i32,i32,f32
2021,10000,500011,0.0
2021,10000,500020,0.0
2021,10000,500046,0.0
2021,10000,500062,0.0
2021,10000,500071,0.0
